In [1]:
import re
import time

import pandas as pd
import numpy as np
import pyarrow.parquet as pq

from collections import Counter

In [2]:
#Константные списки
part_days = ['evening', 'night', 'day', 'morning']
week_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday', 'Sunday' ]

## Загрузка данных

### Считываем данные с признаками последовательно из отдельных паркетных файлов и сразу же агрегируем их по user_id

In [3]:
#Data with features
#Creating DF with agregate information from all files
first_all_users = 0
top_urls = []
data_files = ['parcet_0', 'parcet_1', 'parcet_2', 'parcet_3', 'parcet_4', 'parcet_5', 'parcet_6', 'parcet_7', 'parcet_8', 'parcet_9']


START = time.time()
print(f'START creating in {time.strftime("%H:%M:%S", time.localtime())}')
for data_file in data_files:
    start = time.time()
    data = pd.read_parquet(data_file)
    end = time.time()
    use_time = (end - start)
    print(f"Data file {data_file}.Data read. Used {use_time} sec.")
    
    start = time.time()
    data['date'] = pd.to_datetime(data['date'])
    data['week_day'] = data.date.dt.day_name()
    end = time.time()
    use_time = (end - start)
    print(f"Data file {data_file}.Week days calc. Used {use_time} sec.")
   
  
    start = time.time()
    data['region_name'] = data.region_name.transform(lambda x: '_'.join(x.split(" ")))
    data['city_name'] = data.city_name.transform(lambda x: '_'.join(x.split(" ")))
    data['cpe_model_name'] = data.cpe_model_name.transform(lambda x: '_'.join(x.split(" ")))
    data['cpe_type_cd'] = data['cpe_type_cd'] + ' ' 
    data['region_name'] = data['region_name'] + ' ' 
    data['city_name'] = data['city_name'] + ' ' 
    data['cpe_model_name'] = data['cpe_model_name'] + ' '
    data['week_day'] = data['week_day'] + ' ' 
    data['week_day'] = data['week_day'] *data['request_cnt']
    data['part_of_day'] = data['part_of_day'] + ' '
    data['url_host'] = data['part_of_day'] + data['url_host'] + ' '
    data['part_of_day'] = data['part_of_day']*data['request_cnt']
    end = time.time()
    use_time = (end - start)
    print(f"Data file {data_file}.reparing strings finished. Used {use_time} sec.")
     
    start = time.time()
    users = data.groupby(by = 'user_id', as_index = False).agg(\
                                                           cpe_type_cd = ('cpe_type_cd', sum), region_name = ('region_name', sum), \
                                                           city_name = ('city_name', sum), cpe_model_name = ('cpe_model_name', sum),\
                                                          begin_date = ('date', min), end_date = ('date', max), week_days = ('week_day', sum),\
                                                           part_of_day = ('part_of_day', sum), url_host = ('url_host', sum), price = ('price', 'mean'))
    
    del data
    end = time.time()
    use_time = (end - start)
    print(f"Data file {data_file}.Grouping finished. Prom users df create. Used {use_time} sec.")
    
    start = time.time()
    users['region_name'] = users.region_name.transform(lambda x: ' '.join(set(x.split(" "))))
    users['city_name'] = users.city_name.transform(lambda x: ' '.join(set(x.split(" "))))   
    users['cpe_model_name'] = users.cpe_model_name.transform(lambda x: ' '.join(set(x.split(" ")))) 
    users['cpe_type_cd'] = users.cpe_type_cd.transform(lambda x: ' '.join(set(x.split(" "))))
    end = time.time()
    use_time = (end - start)
    print(f"Data file {data_file}. Deleting dublicates finished. Used {use_time} sec.")
    
    
    start = time.time()
    if first_all_users == 0:
        all_users = users
        first_all_users = 1
    else:
        all_users = pd.concat([all_users, users])
    end = time.time()
    use_time = (end - start)
    print(f"Data file {data_file}.Grouping finished. All_users upgrate. Used {use_time} sec.") 
    
    del users
    
    start = time.time()
    #all_users.to_csv('all_users')
    end = time.time()
    use_time = (end - start)
    print(f"Data file {data_file}.Grouping finished. Csv-file upgrate. Used {use_time} sec.") 


END = time.time()
use_time = (end - start)   
print(f'FINISHED creating in {time.strftime("%H:%M:%S", time.localtime())}. Summery used time {use_time} sec.')

START creating in 23:48:50
Data file parcet_0.Data read. Used 16.886138200759888 sec.
Data file parcet_0.Week days calc. Used 13.027215480804443 sec.
Data file parcet_0.reparing strings finished. Used 85.56792116165161 sec.
Data file parcet_0.Grouping finished. Prom users df create. Used 249.3321213722229 sec.
Data file parcet_0. Deleting dublicates finished. Used 16.442820072174072 sec.
Data file parcet_0.Grouping finished. All_users upgrate. Used 0.00099945068359375 sec.
Data file parcet_0.Grouping finished. Csv-file upgrate. Used 0.0 sec.
Data file parcet_1.Data read. Used 17.030155897140503 sec.
Data file parcet_1.Week days calc. Used 13.055615663528442 sec.
Data file parcet_1.reparing strings finished. Used 82.79020857810974 sec.
Data file parcet_1.Grouping finished. Prom users df create. Used 246.77850675582886 sec.
Data file parcet_1. Deleting dublicates finished. Used 16.604637384414673 sec.
Data file parcet_1.Grouping finished. All_users upgrate. Used 1.6166026592254639 sec.
D

In [4]:
all_users.isna().sum()

user_id               0
cpe_type_cd           0
region_name           0
city_name             0
cpe_model_name        0
begin_date            0
end_date              0
week_days             0
part_of_day           0
url_host              0
price             10703
dtype: int64

## Предобработка данных

In [5]:
#Calculation statistics for user_id in aggregate DF
#Сount rows for each part of day for each user_id. Write in new columns ('evening', 'night', 'day', 'morning')
for part in part_days:
    all_users[part] = all_users.part_of_day.transform(lambda x: Counter(str(x).split())[part]) 

#Сount rows for each day of week for each user_id. Write in new columns ('Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday', 'Saturday', 'Sunday')    
for day in week_days:
    all_users[day] = all_users.week_days.transform(lambda x: Counter(str(x).split())[day]) 

#Calc most frequency town, region and model cpe for each user_id. Write in a new columns ('city', 'region', 'cpe').
all_users['city']  = all_users['city_name'].transform(lambda x: Counter(str(x).split()).most_common(1)[0][0])   
all_users['region']  = all_users['region_name'].transform(lambda x: Counter(str(x).split()).most_common(1)[0][0]) 
all_users['cpe']  = all_users['cpe_model_name'].transform(lambda x: Counter(str(x).split()).most_common(1)[0][0])  

#Calc sum count of citys, regions and models cpe for each user_id. Write in a new columns ('city_count', 'region_count').
all_users['city_count']  = all_users['city_name'].transform(lambda x: len(Counter(str(x).split()))) 
all_users['region_count']  = all_users['region_name'].transform(lambda x: len(Counter(str(x).split())))
all_users['cpe_count']  = all_users['cpe_model_name'].transform(lambda x: len(Counter(str(x).split()))) 
    
#Drop unusefull columns    
all_users = all_users.drop(['part_of_day', 'week_days'], axis = 1)
all_users = all_users.drop(['region_name','city_name'], axis = 1)
all_users = all_users.drop(['cpe_model_name'], axis = 1)

In [6]:
all_users.isna().sum()

user_id             0
cpe_type_cd         0
begin_date          0
end_date            0
url_host            0
price           10703
evening             0
night               0
day                 0
morning             0
Monday              0
Tuesday             0
Wednesday           0
Thursday            0
Friday              0
Saturday            0
Sunday              0
city                0
region              0
cpe                 0
city_count          0
region_count        0
cpe_count           0
dtype: int64

In [7]:
#Save prepared data ibn a file
all_users.to_csv('all_data_prepared_3')

In [8]:
#Clear memory 
del all_users